<a href="https://www.kaggle.com/code/arshadulhoque/aes-2-train?scriptVersionId=181474428" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
import torch.nn as nn
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from sklearn.metrics import cohen_kappa_score, classification_report, confusion_matrix, f1_score

In [ ]:
data = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
data

In [ ]:
# Remove essay_id column
data.drop('essay_id', axis=1, inplace=True)

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['score'], random_state=42)

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")

In [ ]:
# Load model directly
from transformers import AutoModel
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-xsmall", num_labels=6)

In [ ]:
# Tokenize and encode the training data
train_encodings = tokenizer(list(train_data['full_text']), truncation=True, padding=True, max_length=1024, return_tensors='pt')
train_labels = torch.tensor(train_data['score'].values - 1)  # Adjust labels to start from 0

In [ ]:
# Tokenize and encode the testing data
test_encodings = tokenizer(list(test_data['full_text']), truncation=True, padding=True, max_length=1024, return_tensors='pt')
test_labels = torch.tensor(test_data['score'].values - 1)  # Adjust labels to start from 0

In [ ]:
# Create DataLoader for training and testing sets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  model = nn.DataParallel(model)

model.to(device)

In [ ]:
accumulation_steps = 4  # Accumulate gradients over 4 steps
total_steps = len(train_loader) * accumulation_steps

for epoch in range(1):  # Number of epochs
    model.train()
    optimizer.zero_grad()
    step = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device, dtype=torch.long)  # Ensure inputs are of type torch.long
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Perform backpropagation for gradient accumulation
        loss = loss / accumulation_steps
        loss.backward()

        step += 1
        if step % accumulation_steps == 0 or step == total_steps:
            optimizer.step()
            optimizer.zero_grad()

In [ ]:
# Evaluation on test set
model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device, dtype=torch.long)  # Ensure inputs are of type torch.long
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        preds = logits.argmax(dim=1).cpu().numpy()
        test_preds.extend(preds)
        test_labels.extend(labels.cpu().numpy())

# Calculate Quadratic Weighted Kappa score
qwk_score = cohen_kappa_score(test_labels, test_preds, weights='quadratic')
print(f"Quadratic Weighted Kappa Score: {qwk_score:.4f}")

In [ ]:
tokenizer.save_pretrained("/kaggle/working/trained_deberta_tokenizer")
model.save_pretrained("/kaggle/working/trained_deberta_model")